In [1]:
from tfx import v1 as tfx
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

2023-02-17 13:24:09.180690: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.11.0
TFX version: 1.12.0


In [2]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import pipeline_settings as settings
from components.example_generator import get_example_gen
from tfx.components.example_gen.import_example_gen.component import ImportExampleGen

interactive_context = InteractiveContext(
    pipeline_name=settings.PIPELINE_NAME,
    pipeline_root=settings.PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(
        settings.METADATA_PATH)
)
example_gen = get_example_gen()
# example_gen = tfx.dsl.Resolver(
#     strategy_class=tfx.dsl.experimental.LatestArtifactStrategy,
#     examples=tfx.dsl.Channel(
#         type=tfx.types.standard_artifacts.Examples)  #  , producer_component_id=example_gen.id)
# ).with_id('ExampleResolver_LatestArtifactStrategy')

interactive_context.run(example_gen)


  0%|          | 0/70000 [00:00<?, ?it/s]2023-02-17 13:24:18.250822: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: UNKNOWN ERROR (34)
2023-02-17 13:24:18.250923: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a99aa484eb14): /proc/driver/nvidia/version does not exist
2023-02-17 13:24:18.251372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████| 70000/70000 [00:21<00:00, 3324.41it/s]


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 263
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [3]:
from components.statistics_generator import get_statistics_gen
statistics_gen = get_statistics_gen(example_gen)
interactive_context.run(statistics_gen)
interactive_context.show(statistics_gen.outputs['statistics'])


In [4]:
from components.schema_generator import get_schema_gen
schema_gen = get_schema_gen(statistics_gen)
interactive_context.run(schema_gen)
interactive_context.show(schema_gen.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'image',BYTES,required,,-
'label',INT,required,,-


In [5]:
from tensorflow_transform.tf_metadata import schema_utils
from tfx.types import artifact_utils
from tfx.utils import io_utils

# schema_uri = schema_gen.outputs["schema"].get()[0].uri + '/schema.pbtxt'
# schema_debug = tfdv.load_schema_text(schema_uri)
schema_debug = io_utils.SchemaReader().read(
    io_utils.get_only_uri_in_dir(
        artifact_utils.get_single_uri(schema_gen.outputs['schema'].get())))
feature_spec = schema_utils.schema_as_feature_spec(schema_debug).feature_spec

print(feature_spec)


{'image': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None), 'label': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None)}


In [6]:
print(example_gen.outputs['examples'].get()[0].split_names)
dataset_train_uri = io_utils.get_only_uri_in_dir(artifact_utils.get_split_uri(
    example_gen.outputs['examples'].get(), 'train'))
dataset_train_debug = tf.data.TFRecordDataset(dataset_train_uri, compression_type='GZIP')

dataset_eval_uri = io_utils.get_only_uri_in_dir(artifact_utils.get_split_uri(
    example_gen.outputs['examples'].get(), 'eval'))
dataset_eval_debug = tf.data.TFRecordDataset(
    dataset_eval_uri, compression_type='GZIP')
# Create a batch from the dataset
for records in dataset_train_debug.batch(1).take(1):

  # Parse the batch to get a dictionary of raw features
  parsed_examples = tf.io.parse_example(records, feature_spec)
  # Print the results
  print("\nRAW FEATURES:")
  for key, value in parsed_examples.items():
    print(f'{key}: {value.numpy()}')


["train", "eval"]

RAW FEATURES:
image: [[b'\x08\x04\x12\x08\x12\x02\x08\x1c\x12\x02\x08\x1c"\x90\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\rI\x00\x00\x01\x04\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00$\x88\x7f>6\x00\x00\x00\x01\x03\x04\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x00f\xcc\xb0\x86\x90{\x17\x00\x00\x00\x00\x0c\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x9b\xec\xcf\xb2k\x9c\xa1m@\x17M\x82H\x0f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00E\xcf\xdf\xda\xd8\xd8\xa3\x7fyz\x92\x8dX\xacB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x01\x00\xc

In [7]:
from tfx.components.util import tfxio_utils
from tfx_bsl.public import tfxio

_IMAGE_KEY = 'image'


def image_decoder_debug(input, output):
    numpy_image = tf.io.parse_tensor(
        input[_IMAGE_KEY][:][0], out_type=tf.uint8)
    transformed_input = {}
    transformed_input[_IMAGE_KEY] = numpy_image
    return input, output

data_accessor = tfx.components.DataAccessor(
    tf_dataset_factory=tfxio_utils.get_tf_dataset_factory_from_artifact(
        [tfx.types.standard_artifacts.Examples()], []),
    record_batch_factory=None,
    data_view_decode_fn=None)

tf_dataset_debug = data_accessor.tf_dataset_factory(
    dataset_train_uri,
    tfxio.TensorFlowDatasetOptions(
        batch_size=1, label_key='label'),
    schema=schema_debug).map(image_decoder_debug).batch(20).repeat()

# for records in tf_dataset_debug.take(1):
#     print(records)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [8]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    materialize=False,
    module_file="components/trainer.py")

interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying evaluator.py -> build/lib
copying example_generator.py -> build/lib
copying model_resolver.py -> build/lib
copying model_settings.py -> build/lib
copying pusher.py -> build/lib
copying schema_generator.py -> build/lib
copying statistics_generator.py -> build/lib
copying trainer.py -> build/lib
installing to /tmp/tmpvjbjajdm
running install
running install_lib
copying build/lib/schema_generator.py -> /tmp/tmpvjbjajdm
copying build/lib/model_settings.py -> /tmp/tmpvjbjajdm
copying build/lib/example_generator.py -> /tmp/tmpvjbjajdm
copying build/lib/statistics_generator.py -> /tmp/tmpvjbjajdm
copying build/lib/evaluator.py -> /tmp/tmpvjbjajdm
copying build/lib/pusher.py -> /tmp/tmpvjbjajdm
copying build/lib/trainer.py -> /tmp/tmpvjbjajdm
copying build/lib/model_resolver.py -> /tmp/tmpvjbjajdm
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_

/usr/local/lib/python3.8/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpvjbjajdm/tfx_user_code_Transform-0.0+e2e2d25f5ac0db521058fff2b88c5ecfe7a2c13419946c77665ecb45564ce830-py3.8.egg-info
running install_scripts
creating /tmp/tmpvjbjajdm/tfx_user_code_Transform-0.0+e2e2d25f5ac0db521058fff2b88c5ecfe7a2c13419946c77665ecb45564ce830.dist-info/WHEEL
creating '/tmp/tmpliruglk0/tfx_user_code_Transform-0.0+e2e2d25f5ac0db521058fff2b88c5ecfe7a2c13419946c77665ecb45564ce830-py3-none-any.whl' and adding '/tmp/tmpvjbjajdm' to it
adding 'evaluator.py'
adding 'example_generator.py'
adding 'model_resolver.py'
adding 'model_settings.py'
adding 'pusher.py'
adding 'schema_generator.py'
adding 'statistics_generator.py'
adding 'trainer.py'
adding 'tfx_user_code_Transform-0.0+e2e2d25f5ac0db521058fff2b88c5ecfe7a2c13419946c77665ecb45564ce830.dist-info/METADATA'
adding 'tfx_user_code_Transform-

Processing /root/tfx_data/minst_pipeline_root/_wheels/tfx_user_code_Transform-0.0+e2e2d25f5ac0db521058fff2b88c5ecfe7a2c13419946c77665ecb45564ce830-py3-none-any.whl


Processing /root/tfx_data/minst_pipeline_root/_wheels/tfx_user_code_Transform-0.0+e2e2d25f5ac0db521058fff2b88c5ecfe7a2c13419946c77665ecb45564ce830-py3-none-any.whl


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 266
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [9]:
interactive_context.show(transform.outputs['post_transform_schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'image',FLOAT,required,,-
'label',INT,required,,-


In [10]:
# Uses user-provided Python function that trains a model.
trainer = tfx.components.Trainer(
    module_file="components/trainer.py",
    examples=example_gen.outputs['examples'],

    # NEW: Pass transform_graph to the trainer.
    transform_graph=transform.outputs['transform_graph'],

    train_args=tfx.proto.TrainArgs(num_steps=1000),
    eval_args=tfx.proto.EvalArgs(num_steps=500))
interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying evaluator.py -> build/lib
copying example_generator.py -> build/lib
copying model_resolver.py -> build/lib
copying model_settings.py -> build/lib
copying pusher.py -> build/lib
copying schema_generator.py -> build/lib
copying statistics_generator.py -> build/lib
copying trainer.py -> build/lib
installing to /tmp/tmp00f_dc_n
running install
running install_lib


/usr/local/lib/python3.8/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


copying build/lib/schema_generator.py -> /tmp/tmp00f_dc_n
copying build/lib/model_settings.py -> /tmp/tmp00f_dc_n
copying build/lib/example_generator.py -> /tmp/tmp00f_dc_n
copying build/lib/statistics_generator.py -> /tmp/tmp00f_dc_n
copying build/lib/evaluator.py -> /tmp/tmp00f_dc_n
copying build/lib/pusher.py -> /tmp/tmp00f_dc_n
copying build/lib/trainer.py -> /tmp/tmp00f_dc_n
copying build/lib/model_resolver.py -> /tmp/tmp00f_dc_n
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp00f_dc_n/tfx_user_code_Train

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


1000/1000 [==============================] - 8s 7ms/step - loss: 0.9915 - sparse_categorical_accuracy: 0.6161 - val_loss: 0.8009 - val_sparse_categorical_accuracy: 0.6886


INFO:tensorflow:Assets written to: /root/tfx_data/minst_pipeline_root/Trainer/model/267/Format-Serving/assets


INFO:tensorflow:Assets written to: /root/tfx_data/minst_pipeline_root/Trainer/model/267/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 267
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))